In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import time

# ==========================================
# 1. Metrics
# ==========================================
def calculate_metrics(G):
    degrees = np.array([d for n, d in G.degree()])
    if len(degrees) == 0: return 0, 0, 0, 0, 0, 0
    
    n = len(degrees)
    total_deg = np.sum(degrees)
    
    # 1. Nakamoto
    sorted_deg = np.sort(degrees)[::-1]
    current_sum = 0
    nakamoto = 0
    for d in sorted_deg:
        current_sum += d
        nakamoto += 1
        if current_sum > total_deg * 0.51:
            break
            
    # 2. Gini
    sorted_asc = np.sort(degrees)
    cum_deg = np.cumsum(sorted_asc)
    if cum_deg[-1] == 0: gini = 0
    else: gini = (n + 1 - 2 * np.sum(cum_deg) / cum_deg[-1]) / n
        
    # 3. HHI
    shares = degrees / total_deg if total_deg > 0 else degrees
    hhi = np.sum(shares ** 2)

    # 4. Degree Centrality
    deg_cent = nx.degree_centrality(G)
    max_deg_c = max(deg_cent.values()) if deg_cent else 0

    # 5. Betweenness Centrality
    k_sample = int(n * 0.1) if n > 100 else None 
    bet_cent = nx.betweenness_centrality(G, k=k_sample) 
    max_bet_c = max(bet_cent.values()) if bet_cent else 0

    # 6. Closeness Centrality
    clo_cent = nx.closeness_centrality(G)
    max_clo_c = max(clo_cent.values()) if clo_cent else 0
    
    return nakamoto, gini, hhi, max_deg_c, max_bet_c, max_clo_c

# ==========================================
# 2. simulation process
# ==========================================
final_size = 300  
m_edges = 2       

G_centralized = nx.complete_graph(5)
G_decentralized = nx.complete_graph(5)

history = {
    'time': [],
    'A_nakamoto': [], 'A_gini': [], 'A_hhi': [],
    'B_nakamoto': [], 'B_gini': [], 'B_hhi': [],
    'A_deg_c': [], 'A_bet_c': [], 'A_clo_c': [],
    'B_deg_c': [], 'B_bet_c': [], 'B_clo_c': []
}

print("Starting simulation...")
for t in range(5, final_size):
    # Scenario A
    degrees_A = [d for n, d in G_centralized.degree()]
    probs_A = np.array(degrees_A) / sum(degrees_A)
    targets_A = np.random.choice(G_centralized.nodes(), size=m_edges, p=probs_A, replace=False)
    new_node_A = max(G_centralized.nodes()) + 1
    G_centralized.add_node(new_node_A)
    for target in targets_A: G_centralized.add_edge(new_node_A, target)

    # Scenario B
    existing_nodes_B = list(G_decentralized.nodes())
    targets_B = random.sample(existing_nodes_B, m_edges)
    new_node_B = max(G_decentralized.nodes()) + 1
    G_decentralized.add_node(new_node_B)
    for target in targets_B: G_decentralized.add_edge(new_node_B, target)
        
    if t % 20 == 0 or t == final_size - 1:
        nk_A, gini_A, hhi_A, dc_A, bc_A, cc_A = calculate_metrics(G_centralized)
        nk_B, gini_B, hhi_B, dc_B, bc_B, cc_B = calculate_metrics(G_decentralized)
        
        history['time'].append(t)
        history['A_nakamoto'].append(nk_A); history['A_gini'].append(gini_A); history['A_hhi'].append(hhi_A)
        history['A_deg_c'].append(dc_A); history['A_bet_c'].append(bc_A); history['A_clo_c'].append(cc_A)
        history['B_nakamoto'].append(nk_B); history['B_gini'].append(gini_B); history['B_hhi'].append(hhi_B)
        history['B_deg_c'].append(dc_B); history['B_bet_c'].append(bc_B); history['B_clo_c'].append(cc_B)

print("Simulation finished. Saving 9 separate images...")

x_label = "Block Height (Time)"

figsize_single = (8, 5)

# ==========================================
# save pngs
# ==========================================

# --- 1. Nakamoto ---
plt.figure(figsize=figsize_single)
plt.plot(history['time'], history['A_nakamoto'], 'r--', linewidth=2, label='Scenario A (Pools)')
plt.plot(history['time'], history['B_nakamoto'], 'g-', linewidth=2, label='Scenario B (P2P)')
plt.title("Security: Nakamoto Coefficient", fontsize=14, fontweight='bold')
plt.ylabel("Min Nodes for 51% Attack")
plt.xlabel(x_label)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('1_Nakamoto.png', dpi=300)
plt.close()

# --- 2. Gini ---
plt.figure(figsize=figsize_single)
plt.plot(history['time'], history['A_gini'], 'r--', linewidth=2, label='Scenario A (Pools)')
plt.plot(history['time'], history['B_gini'], 'g-', linewidth=2, label='Scenario B (P2P)')
plt.title("Inequality: Gini Coefficient", fontsize=14, fontweight='bold')
plt.ylabel("Inequality (0-1)")
plt.xlabel(x_label)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('2_Gini.png', dpi=300)
plt.close()

# --- 3. HHI ---
plt.figure(figsize=figsize_single)
plt.plot(history['time'], history['A_hhi'], 'r--', linewidth=2, label='Scenario A (Pools)')
plt.plot(history['time'], history['B_hhi'], 'g-', linewidth=2, label='Scenario B (P2P)')
plt.title("Concentration: HHI Index", fontsize=14, fontweight='bold')
plt.ylabel("Market Concentration")
plt.xlabel(x_label)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('3_HHI.png', dpi=300)
plt.close()

# --- 4. Max Degree (Whale Factor) ---
plt.figure(figsize=figsize_single)
plt.plot(history['time'], history['A_deg_c'], 'r--', linewidth=2, label='Scenario A (Pools)')
plt.plot(history['time'], history['B_deg_c'], 'g-', linewidth=2, label='Scenario B (P2P)')
plt.title("The Whale Factor (Max Degree)", fontsize=14, fontweight='bold')
plt.ylabel("Share of Total Connections")
plt.xlabel(x_label)
plt.annotate('Super-Node Formation', xy=(50, 0.4), xytext=(100, 0.5),
             arrowprops=dict(facecolor='black', shrink=0.05), fontsize=10)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('4_Max_Degree.png', dpi=300)
plt.close()

# --- 5. Max Betweenness ---
plt.figure(figsize=figsize_single)
plt.plot(history['time'], history['A_bet_c'], 'r--', linewidth=2, label='Scenario A (Pools)')
plt.plot(history['time'], history['B_bet_c'], 'g-', linewidth=2, label='Scenario B (P2P)')
plt.title("Gatekeeper Power (Betweenness)", fontsize=14, fontweight='bold')
plt.ylabel("Control of Flow")
plt.xlabel(x_label)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('5_Betweenness.png', dpi=300)
plt.close()

# --- 6. Max Closeness ---
plt.figure(figsize=figsize_single)
plt.plot(history['time'], history['A_clo_c'], 'r--', linewidth=2, label='Scenario A (Pools)')
plt.plot(history['time'], history['B_clo_c'], 'g-', linewidth=2, label='Scenario B (P2P)')
plt.title("Propagation Speed (Closeness)", fontsize=14, fontweight='bold')
plt.ylabel("Spreading Efficiency")
plt.xlabel(x_label)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('6_Closeness.png', dpi=300)
plt.close()

# --- Helper Function for Topology ---
def save_topology_graph(G, filename, title, color_theme):
    plt.figure(figsize=(8, 8)) # 方形画布适合网络图
    degrees = dict(G.degree())
    whale_node = max(degrees, key=degrees.get)
    max_deg = degrees[whale_node]
    
    pos = nx.spring_layout(G, seed=42, k=0.15)
    
    # 1. Miners
    node_sizes = [v * 5 + 20 for v in degrees.values()]
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, 
                           node_color=color_theme, alpha=0.6)
    
    # 2. Edges
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.2, width=0.5)
    
    # 3. Whale Highlight
    avg_deg = np.mean(list(degrees.values()))
    if max_deg > avg_deg * 2:
        nx.draw_networkx_nodes(G, pos, nodelist=[whale_node], 
                               node_size=max_deg*10, node_color='gold', edgecolors='black')
        x, y = pos[whale_node]
        plt.annotate('Dominant Mining Pool\n(The Whale)', xy=(x, y), xytext=(x+0.3, y+0.3),
                    arrowprops=dict(facecolor='black', shrink=0.05),
                    fontsize=14, fontweight='bold', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", alpha=0.8))

    plt.title(title, fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()

# --- 7. Topology A (Centralized) ---
save_topology_graph(G_centralized, '7_Topology_Centralized.png', 
                    "Scenario A: Centralized Mining Pool\n(Note the 'Whale' in Gold)", 'red')

# --- 8. Topology B (Decentralized) ---
save_topology_graph(G_decentralized, '8_Topology_Decentralized.png', 
                    "Scenario B: Distributed Mesh\n(No Single Point of Failure)", 'green')

# --- 9. Summary Text Image ---
plt.figure(figsize=(8, 4))
summary_text = (
    "SIMULATION VERDICT:\n"
    "===================\n\n"
    "SCENARIO A (Mining Pools):\n"
    "--------------------------\n"
    "• Visual: A 'Whale' (Gold Node) emerges.\n"
    "• Logic: New miners join the biggest pool\n"
    "  for stable rewards (Rich-get-Richer).\n"
    "• Risk: High Censorship Risk.\n\n"
    "SCENARIO B (Ideal P2P):\n"
    "-----------------------\n"
    "• Visual: Flat structure, no King.\n"
    "• Logic: Random peer discovery protocol.\n"
    "• Result: High censorship resistance."
)
plt.text(0.01, 0.5, summary_text, fontsize=12, family='monospace', va='center', 
         bbox=dict(boxstyle="round,pad=0.5", fc="#f0f0f0", ec="gray"))
plt.axis('off')
plt.tight_layout()
plt.savefig('9_Summary_Text.png', dpi=300)
plt.close()

print("All 9 images have been saved individually.")

Starting simulation...
Simulation finished. Saving 9 separate images...
All 9 images have been saved individually.
